In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.base import clone
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier
import warnings

from AdaSample import AdaSample

In [2]:
HideFrac = 0.8
TrainFrac = 0.6
N = 200000

SampFrac = 0.1 #Subsampling of training data when preforming Adasampling
NBoosts = 50 #Number of AdaSampling rounds

X, y = datasets.make_classification(N, 10, 5)

In [3]:
y_PU = np.copy(y)
Ps = np.where(y_PU == 1)[0]
y_PU[np.random.choice(Ps, int(np.floor(len(Ps) * HideFrac)))] = 0

inds = np.random.permutation(X.shape[0])
train_inds, test_inds = inds[:int(N * TrainFrac)], inds[int(N * TrainFrac):]

X_train, X_test = X[train_inds, :], X[test_inds, :]
y_train, y_test = y[train_inds], y[test_inds]
y_PU_train, y_PU_test = y_PU[train_inds], y_PU[test_inds]

In [4]:
clf_res = {}
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for name, clf in [("SGD_lasso", SGDClassifier(loss="log", penalty="l1")),
                      ("RF", RandomForestClassifier()),
                      ("SGD_SVM", SGDClassifier(loss="modified_huber")),
                      ("XGB", XGBClassifier()),
                      ("NB", GaussianNB())]:
        print(name)
        clf_res[name] = {}
        ada = AdaSample(clone(clf))
        ada.fit(X_train, y_PU_train, C=10, sampleFactor=SampFrac, n_rounds=NBoosts)
        probas = ada.predict_proba(X_test, single=True)[:, 1]
        clf_res[name]["AdaSingle"] = accuracy_score(y_test, (probas > 0.5).astype(np.int))

        probas = ada.predict_proba(X_test)[:, 1]
        clf_res[name]["AdaEnsemble"] = accuracy_score(y_test, (probas > 0.5).astype(np.int))

        naive_clf = clone(clf)
        naive_clf.fit(X_train, y_train)
        clf_res[name]["Original"] = naive_clf.score(X_test, y_test)

        uNeg_clf = clone(clf)
        uNeg_clf.fit(X_train, y_PU_train)
        clf_res[name]["Baseline"] = uNeg_clf.score(X_test, y_test)
        
pd.DataFrame.from_dict(clf_res)

  6%|▌         | 3/50 [00:00<00:01, 25.95it/s]

SGD_lasso
Training AdaSamples..


  0%|          | 0/10 [00:00<?, ?it/s]

Training 10 Classifiers


  0%|          | 0/50 [00:00<?, ?it/s]

...

NB
Training AdaSamples..


 10%|█         | 1/10 [00:00<00:00,  9.55it/s]

Training 10 Classifiers


100%|██████████| 10/10 [00:00<00:00, 10.41it/s]


,SGD_lasso,RF,SGD_SVM,XGB,NB
AdaEnsemble,0.714600,0.939462,0.781175,0.899438,0.669825
AdaSingle,0.714625,0.927488,0.780963,0.901275,0.669913
Baseline,0.610363,0.621788,0.533225,0.509563,0.660575
Original,0.749350,0.970463,0.676162,0.926162,0.691887


In [5]:
HideFrac=0.2

y_PU = np.copy(y)
Ps = np.where(y_PU == 1)[0]
y_PU[np.random.choice(Ps, int(np.floor(len(Ps) * HideFrac)))] = 0

inds = np.random.permutation(X.shape[0])
train_inds, test_inds = inds[:int(N * TrainFrac)], inds[int(N * TrainFrac):]

X_train, X_test = X[train_inds, :], X[test_inds, :]
y_train, y_test = y[train_inds], y[test_inds]
y_PU_train, y_PU_test = y_PU[train_inds], y_PU[test_inds]

In [6]:
clf_res = {}
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for name, clf in [("SGD_lasso", SGDClassifier(loss="log", penalty="l1")),
                      ("RF", RandomForestClassifier()),
                      ("SGD_SVM", SGDClassifier(loss="modified_huber")),
                      ("XGB", XGBClassifier()),
                      ("NB", GaussianNB())]:
        print(name)
        clf_res[name] = {}
        ada = AdaSample(clone(clf))
        ada.fit(X_train, y_PU_train, C=10, sampleFactor=SampFrac, n_rounds=NBoosts)
        probas = ada.predict_proba(X_test, single=True)[:, 1]
        clf_res[name]["AdaSingle"] = accuracy_score(y_test, (probas > 0.5).astype(np.int))

        probas = ada.predict_proba(X_test)[:, 1]
        clf_res[name]["AdaEnsemble"] = accuracy_score(y_test, (probas > 0.5).astype(np.int))

        naive_clf = clone(clf)
        naive_clf.fit(X_train, y_train)
        clf_res[name]["Original"] = naive_clf.score(X_test, y_test)

        uNeg_clf = clone(clf)
        uNeg_clf.fit(X_train, y_PU_train)
        clf_res[name]["Baseline"] = uNeg_clf.score(X_test, y_test)

pd.DataFrame.from_dict(clf_res)

  8%|▊         | 4/50 [00:00<00:01, 32.25it/s]

SGD_lasso
Training AdaSamples..


  0%|          | 0/10 [00:00<?, ?it/s]

Training 10 Classifiers


  0%|          | 0/50 [00:00<?, ?it/s]

...

NB
Training AdaSamples..


 20%|██        | 2/10 [00:00<00:00, 13.69it/s]

Training 10 Classifiers


100%|██████████| 10/10 [00:00<00:00, 13.68it/s]


,SGD_lasso,RF,SGD_SVM,XGB,NB
AdaEnsemble,0.727137,0.960425,0.783537,0.907763,0.666275
AdaSingle,0.727087,0.955113,0.782212,0.907488,0.666375
Baseline,0.737300,0.933737,0.545837,0.914713,0.764387
Original,0.728313,0.970975,0.634550,0.927075,0.689675
